In [1]:
# libraries
import pandas as pd
import numpy as np
import time
import torch
from tqdm import tqdm
from preprocessing import extract_data_mooc, extractFeatures,extractItemUserId,extractNextStateItem,extractNextUserState,UserNextInteraction, delta, t_batch_update
from model import RODIE,dynamic_embedding
from train import train_rodie

In [2]:
## Téléchargement des données
!wget https://snap.stanford.edu/data/act-mooc.tar.gz
!tar -xzf  act-mooc.tar.gz
!mkdir data

--2022-03-17 09:23:14--  https://snap.stanford.edu/data/act-mooc.tar.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5378133 (5.1M) [application/x-gzip]
Saving to: ‘act-mooc.tar.gz’

act-mooc.tar.gz     100%[===================>]   5.13M  8.27MB/s    in 0.6s    

2022-03-17 09:23:15 (8.27 MB/s) - ‘act-mooc.tar.gz’ saved [5378133/5378133]



In [4]:
features = pd.read_csv("act-mooc/mooc_action_features.tsv",sep="\t")
labels = pd.read_csv("act-mooc/mooc_action_labels.tsv",sep="\t")
users = pd.read_csv("act-mooc/mooc_actions.tsv",sep="\t")

#### Load & Preprocess Data

In [ ]:
mooc_data = extract_data_mooc()

delta_u  = delta(mooc_data.copy(),"user_id")
delta_i  = delta(mooc_data.copy(),"item_id")
nextItemInteraction = UserNextInteraction(mooc_data.copy())
next_state_user = extractNextUserState(mooc_data.copy())


mooc_data['delta_u'] = delta_u
mooc_data['delta_i'] = delta_i
mooc_data['nextItemInteraction'] = nextItemInteraction
mooc_data['next_state_user'] = next_state_user

data = mooc_data.copy()
data = data[ (data.nextItemInteraction != -1) | (data.next_state_user != -1)  ]

data = data[['user_id', 'item_id', 'timestamp', 'state_label','delta_u', 'delta_i', 'nextItemInteraction', 'next_state_user','f1', 'f2', 'f3','f4']]
data.head()

#### T-batches

In [6]:
t_batches = t_batch_update(data.reset_index())

T-Batch start...
Number of interaction = 404702
T-Batch ends !


##### Initialize Device

In [7]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = "cpu"
print('Using device:', device)

Using device: cpu


##### Initialize the model

In [10]:
embedding_dim = 32
data_torch = torch.from_numpy(data.values.astype(np.float32))
U_dynamic,I_dynamic = dynamic_embedding(data_torch,embedding_dim)  # Initial dynamic embedding
    
U_dynamic = U_dynamic.to(device)
I_dynamic = I_dynamic.to(device)

model = RODIE(embedding_dim,data_torch,device=device).to(device)

Initialisation of dynamic embedding... Done !
Dynamic Embedding shape : Users [7047, 32], 	 Items [97, 32]
Number of users of 7047 
 Number of items 97 

Dataset size [404702, 12]
Initialisation of static embedding... Done !
Static Embedding shape : Users [7047, 7047], 	 Items [97, 97]
Initialisation of rnn's with relu activation function... Done !
Initialisation of MLP... Done !


In [12]:
# Its important to add this to the loss, because the dataset is unbalanced
dropout_ratio = len(data['state_label'])/(1+np.sum(data['state_label']))
weight_ratio = torch.Tensor([1,dropout_ratio]).to(device)

##### Train Loop

In [13]:
# Test le modèle sur peu de données
import itertools
t_batches_ = dict(itertools.islice(t_batches.items(), 3000))

In [19]:

from torch.nn import MSELoss, HuberLoss,L1Loss,CrossEntropyLoss
from tqdm import tqdm
import torch
from  torch import nn
from torch.nn import RNNCell
from torch.nn.functional import one_hot
import math
from torch.nn import MSELoss, HuberLoss,L1Loss,CrossEntropyLoss
from torch.nn import functional as F

def regularizer(actual_user_embedding,future_user_embedding,lambda_u,
                               actual_item_embedding,future_item_embedding,lambda_i
                               ):
    u_regularization_loss =  MSELoss()(actual_user_embedding,future_user_embedding)
    i_regularization_loss =  MSELoss()(actual_item_embedding,future_item_embedding)
    return lambda_u* u_regularization_loss + lambda_i* i_regularization_loss 


def train_rodie(t_batches,
          data,
          U,
          I,
          weight_ratio,
          model,
          optimizer,
          n_epochs,
          lambda_u,
          lambda_i,
          device,

          ):
  print("Training...")
 # U_copy = U.clone().detach()
 # I_copy = I.clone().detach()

  for e in range(n_epochs):
    l = 0
    
    for (_,rows),_ in zip(t_batches.items(),tqdm(range(len(t_batches)), position=0, leave=True)):
      optimizer.zero_grad()
      users_idx,items_idx = extractItemUserId(data,rows)

      state_label,delta_u,delta_i,f = extractFeatures(data,rows)

      next_state,next_item = extractNextStateItem(data,rows)

      u_static, i_static = model.static_users_embedding[users_idx], model.static_items_embedding[items_idx]

      user_embedding, item_embedding = U[users_idx], I[items_idx]
      next_item_static_embedding, next_item_dynamic_embedding = model.static_items_embedding[[int(x) for x in next_item]], I[[int(x) for x in next_item]]

     # next_state = next_state.type(torch.LongTensor).to(device)
      item_embedding = item_embedding.to(device)
      user_embedding  = user_embedding.to(device)
      u_static = u_static.to(device)
      i_static = i_static.to(device)
      f = f.to(device)
      delta_u = delta_u.to(device)
      delta_i = delta_i.to(device)
      next_state = next_state.type(torch.LongTensor).to(device)
      next_item_dynamic_embedding = next_item_dynamic_embedding.to(device)
      next_item_static_embedding = next_item_static_embedding.to(device)

      
      future_user_embedding,future_item_embedding,U_pred_state,j_tilde,j_true  = model(item_embedding,
                user_embedding,
                u_static,
                i_static,
                f,
                delta_u,
                delta_i,
                next_state,
                next_item_dynamic_embedding,
                next_item_static_embedding) 
      U[users_idx] = future_user_embedding.detach().clone()
      I[items_idx] = future_item_embedding.detach().clone()     
      # Return loss value between the predicted embedding "j_tilde" and the real next item embedding j_true
      loss = MSELoss()(j_tilde,j_true)
      loss += regularizer(user_embedding,future_user_embedding,lambda_u,
                            item_embedding,future_item_embedding,lambda_i
                            )
        
      loss += CrossEntropyLoss(weight_ratio)(U_pred_state,next_state)

      #print(I[0])
      loss.backward()
      l += loss.item()
      torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.)
      optimizer.step()
    print(I[0])
    print("Epoch {} Loss {}".format(e,l))
    #print(I[0])
    #print(U[0])
  return model,U,I

In [ ]:
n_epochs = 3
lambda_u = 1e-3
lambda_i = 1e-3
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-5)

model_,U,I = train_rodie(t_batches_,
          data_torch,
          U_dynamic,
          I_dynamic,
          weight_ratio,
          model,
          optimizer,
          n_epochs,
          lambda_u,
          lambda_i,
          device
          )